<a href="https://colab.research.google.com/github/zhangfuyao/Google-colab/blob/classifier/aiMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision.datasets
from torch import nn
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.utils.data import DataLoader

In [ ]:
# 定义超参数 
EPOCH = 10  # 训练整批数据多少次
BATCH_SIZE = 64
LR = 0.01  # 学习率
momentum = 0.5

In [ ]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
train_data= torchvision.datasets.MNIST(root="./mnist",train=True,download=True,transform=transform)
test_data= torchvision.datasets.MNIST(root="./mnist",train=False,download=True,transform=transform)

train_loader=DataLoader(train_data,batch_size=BATCH_SIZE,shuffle=True)
test_loader=DataLoader(test_data,batch_size=BATCH_SIZE,shuffle=False)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(
                kernel_size=2
            ),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(10, 20, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.out = torch.nn.Sequential(
            torch.nn.Linear(320, 50),
            torch.nn.Linear(50, 10),
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        x = self.out(x)
        return x

In [ ]:
model = CNN()  # 实例化模型
criterion = torch.nn.CrossEntropyLoss()  # 交叉熵损失函数
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=momentum)

In [ ]:
def train(epoch):
    running_loss = 0.0  # 这整个epoch的loss清零
    running_total = 0
    running_correct = 0
    program_bar = tqdm(total=len(train_loader), leave=False)
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data
        optimizer.zero_grad()  # 清零

        # 前向传播
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        program_bar.update()

        running_loss += loss.item()
        predicted = torch.argmax(outputs.data, dim=1)
        running_total += inputs.shape[0]
        running_correct += (predicted == target).sum().item()
        if batch_idx % 300==299:  # 不想要每一次都出loss，浪费时间，选择每300次出一个平均损失,和准确率
            print('[%d, %5d]: loss: %.3f , acc: %.2f %%'
                  % (epoch + 1, batch_idx + 1, running_loss / 300, 100 * running_correct / running_total))

            running_loss = 0.0  # 这小批的loss清零
            running_total = 0
            running_correct = 0  # 这小批的acc清零

In [ ]:
if __name__ == '__main__':
    train(EPOCH)


 33%|███▎      | 305/938 [00:10<00:21, 29.06it/s]

[11,   300]: loss: 0.131 , acc: 96.21 %



 64%|██████▍   | 604/938 [00:20<00:11, 29.50it/s]

[11,   600]: loss: 0.111 , acc: 96.61 %



 96%|█████████▋| 904/938 [00:31<00:01, 28.40it/s]

[11,   900]: loss: 0.097 , acc: 96.91 %



100%|█████████▉| 937/938 [00:32<00:00, 28.17it/s]
                                                 